In [1]:
!pip install groq
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [2]:
import time
from datasets import load_dataset
import os
from groq import Groq

In [3]:
ds = load_dataset("ChilleD/SVAMP")
client = Groq(api_key='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')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/675 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/111k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/700 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

Zero_Shot

In [4]:
for i in ds['test']:
    print(i)
    ans = i['Answer']
    ques = i['question_concat']
    print(ans)
    print(ques)
    break

{'ID': 'chal-736', 'Body': 'Winter is almost here and most animals are migrating to warmer countries. There are 41 bird families living near the mountain. If 35 bird families flew away to asia and 62 bird families flew away to africa', 'Question': 'How many more bird families flew away to africa than those that flew away to asia?', 'Equation': '( 62.0 - 35.0 )', 'Answer': '27', 'Type': 'Subtraction', 'question_concat': 'Winter is almost here and most animals are migrating to warmer countries. There are 41 bird families living near the mountain. If 35 bird families flew away to asia and 62 bird families flew away to africa How many more bird families flew away to africa than those that flew away to asia?'}
27
Winter is almost here and most animals are migrating to warmer countries. There are 41 bird families living near the mountain. If 35 bird families flew away to asia and 62 bird families flew away to africa How many more bird families flew away to africa than those that flew away to

In [19]:
def math_solver_gemma(model_name, query):
    system_prompt = {"role": "system", "content": "You will be given a math word problem, your job is to solve this problem and return one numerical answer. The template to answer the query is to have the last line of the output as Answer: <<<Numerical Answer>>>"}
    chat_history = [system_prompt]
    chat_history.append({"role": "user", "content": query})
    response = client.chat.completions.create(model=model_name, messages=chat_history, temperature=0)
    return response.choices[0].message.content

In [26]:
import re

global_counter = 0

def ans_extractor(output):
    global global_counter
    matcha = re.search(r'Answer:\s*([0-9]+)', output)
    if matcha:
        final_answer = matcha.group(1)
        return final_answer
    else:
        global_counter += 1
        return 0


In [7]:
pt = []
gt = []
counter=0
for i in ds['test']:
    counter+=1
    print(counter)
    query = i['question_concat']
    explaination = math_solver_gemma("gemma2-9b-it", query)
    answer = ans_extractor(explaination)
    pt.append(answer)
    gt.append(i['Answer'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
from sklearn.metrics import accuracy_score
print(global_counter)
accuracy = accuracy_score(gt, pt)
print(f'Accuracy: {accuracy:.2f}')

8
Accuracy: 0.84


In [27]:
def math_solver_llama(model_name, query):

    system_prompt = {"role": "system", "content": "You will be given a math word problem, your job is to solve this problem and return one numerical answer. The template to answer the query is to have the last line of the output as Answer: <<<Numerical Answer>>>"}
    chat_history = [system_prompt]
    chat_history.append({"role": "user", "content": query})
    response = client.chat.completions.create(model=model_name, messages=chat_history, temperature=0)
    return response.choices[0].message.content

In [31]:
pt = []
gt = []
counter=0
for i in ds['test']:
    counter+=1
    print(counter)
    query = i['question_concat']
    explaination = math_solver_llama("llama-3.1-8b-instant", query)
    answer = ans_extractor(explaination)
    pt.append(answer)
    gt.append(i['Answer'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [32]:
from sklearn.metrics import accuracy_score
print(global_counter)
accuracy = accuracy_score(gt, pt)
print(f'Accuracy: {accuracy:.2f}')

30
Accuracy: 0.82
